In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=7c81b7756fdc901fd6e505c616356e607ca4db8ae65d6300a14e0cc86dd00327
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
spark

In [4]:
olympics = [("United States", "Gold", 39),
             ("United States", "Silver", 41),
             ("United States", "Bronze", 33),
             ("China", "Gold", 38),
             ("China", "Silver", 32),
             ("China", "Bronze", 19),
             ("Japan", "Gold", 27),
             ("Japan", "Silver", 14),
             ("Japan", "Bronze", 17),
             ("Great Britain", "Gold", 22),
             ("Great Britain", "Silver", 20),
             ("Great Britain", "Bronze", 22)
             ]


col = ['Country', 'medal', 'count']

olympicsdf = spark.createDataFrame(data = olympics,schema = col)

olympicsdf.printSchema()
olympicsdf.show()

root
 |-- Country: string (nullable = true)
 |-- medal: string (nullable = true)
 |-- count: long (nullable = true)

+-------------+------+-----+
|      Country| medal|count|
+-------------+------+-----+
|United States|  Gold|   39|
|United States|Silver|   41|
|United States|Bronze|   33|
|        China|  Gold|   38|
|        China|Silver|   32|
|        China|Bronze|   19|
|        Japan|  Gold|   27|
|        Japan|Silver|   14|
|        Japan|Bronze|   17|
|Great Britain|  Gold|   22|
|Great Britain|Silver|   20|
|Great Britain|Bronze|   22|
+-------------+------+-----+



In [5]:
pivotDF = olympicsdf.groupBy("Country").sum("count")
pivotDF.printSchema()
pivotDF.show(truncate=False)

root
 |-- Country: string (nullable = true)
 |-- sum(count): long (nullable = true)

+-------------+----------+
|Country      |sum(count)|
+-------------+----------+
|United States|113       |
|China        |89        |
|Great Britain|64        |
|Japan        |58        |
+-------------+----------+



In [6]:
pivotDF = olympicsdf.groupBy("Country").pivot("medal").sum("count")
pivotDF.printSchema()
pivotDF.show(truncate=False)

pivotDF1 = olympicsdf.groupBy("Country").pivot("medal",['Gold','Silver']).sum("count")
pivotDF1.show(truncate=False)

root
 |-- Country: string (nullable = true)
 |-- Bronze: long (nullable = true)
 |-- Gold: long (nullable = true)
 |-- Silver: long (nullable = true)

+-------------+------+----+------+
|Country      |Bronze|Gold|Silver|
+-------------+------+----+------+
|Great Britain|22    |22  |20    |
|China        |19    |38  |32    |
|United States|33    |39  |41    |
|Japan        |17    |27  |14    |
+-------------+------+----+------+

+-------------+----+------+
|Country      |Gold|Silver|
+-------------+----+------+
|Great Britain|22  |20    |
|China        |38  |32    |
|United States|39  |41    |
|Japan        |27  |14    |
+-------------+----+------+



In [7]:
from pyspark.sql.functions import expr
unpivotExpr = "stack(3, 'G', Gold, 'S', Silver, 'B', Bronze) as (medal,Total)"
unPivotDF = pivotDF.select("Country", expr(unpivotExpr))
unPivotDF.show()

+-------------+-----+-----+
|      Country|medal|Total|
+-------------+-----+-----+
|Great Britain|    G|   22|
|Great Britain|    S|   20|
|Great Britain|    B|   22|
|        China|    G|   38|
|        China|    S|   32|
|        China|    B|   19|
|United States|    G|   39|
|United States|    S|   41|
|United States|    B|   33|
|        Japan|    G|   27|
|        Japan|    S|   14|
|        Japan|    B|   17|
+-------------+-----+-----+

